# 1. Install Dependencies

In [ ]:
# Install required libraries
!pip install -U datasets transformers evaluate optuna peft bitsandbytes
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Preprocess Data

In [ ]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)

train_dataset = imdb['train'].shuffle(seed=42)
test_dataset = imdb['test'].shuffle(seed=42)

"""
# For hyperparameter tuning
val_split_ratio = 0.3
val_size = int(len(imdb["train"]) * val_split_ratio)

val_dataset_ht = train_dataset.select([i for i in list(range(val_size))])
train_dataset_ht = train_dataset.select([i for i in list(range(val_size, len(imdb["train"])))])

print(len(test_dataset))
print(len(val_dataset_ht))
print(len(train_dataset_ht))
"""

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


'\n# For hyperparameter tuning\nval_split_ratio = 0.3\nval_size = int(len(imdb["train"]) * val_split_ratio)\n\nval_dataset_ht = train_dataset.select([i for i in list(range(val_size))])\ntrain_dataset_ht = train_dataset.select([i for i in list(range(val_size, len(imdb["train"])))])\n\nprint(len(test_dataset))\nprint(len(val_dataset_ht))\nprint(len(train_dataset_ht))\n'

In [ ]:
from datasets import load_dataset
import torch
import optuna
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import evaluate

torch.manual_seed(42)

CHECKPOINT = "microsoft/deberta-v3-base"
save_path = "./deberta-v3"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)




tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

#3. LoRA

In [ ]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

model

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_proj", "key_proj", "value_proj"],
    modules_to_save=["classifier", "pooler"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

In [ ]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
    #bf16=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss
500,0.178700
1000,0.175000
1500,0.164600


TrainOutput(global_step=1564, training_loss=0.17288431852979733, metrics={'train_runtime': 1235.2457, 'train_samples_per_second': 40.478, 'train_steps_per_second': 1.266, 'total_flos': 1.33055337426816e+16, 'train_loss': 0.17288431852979733, 'epoch': 2.0})


{'eval_loss': 0.14567483961582184, 'eval_accuracy': 0.94688, 'eval_f1': 0.9472220014307289, 'eval_recall': 0.95336, 'eval_precision': 0.9411625335649977, 'eval_runtime': 264.5939, 'eval_samples_per_second': 94.484, 'eval_steps_per_second': 2.955, 'epoch': 2.0}


# 4. Prompt Tuning


In [ ]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 185,031,172 || trainable%: 0.3283


In [ ]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss
500,0.689600
1000,0.679600
1500,0.676700


TrainOutput(global_step=1564, training_loss=0.6818360477457266, metrics={'train_runtime': 764.229, 'train_samples_per_second': 65.425, 'train_steps_per_second': 2.047, 'total_flos': 1.32376327319424e+16, 'train_loss': 0.6818360477457266, 'epoch': 2.0})


{'eval_loss': 0.6920531392097473, 'eval_accuracy': 0.5018, 'eval_f1': 0.00749063670411985, 'eval_recall': 0.00376, 'eval_precision': 0.9591836734693877, 'eval_runtime': 124.7744, 'eval_samples_per_second': 200.362, 'eval_steps_per_second': 6.267, 'epoch': 2.0}


In [ ]:
from transformers import EarlyStoppingCallback
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

early_stopping = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)

best_batch_size = 32

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=1e-3,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=20,
    load_best_model_at_end=True,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


Step,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
500,0.618100,0.320413,0.868280,0.862946,0.829360,0.899367
1000,0.310900,0.177887,0.931320,0.932833,0.953840,0.912731
1500,0.211900,0.165622,0.939320,0.941308,0.973200,0.911441
2000,0.183100,0.133893,0.951800,0.951390,0.943360,0.959557
2500,0.162700,0.123274,0.953440,0.953514,0.955040,0.951994
3000,0.157400,0.124339,0.953880,0.954230,0.961520,0.947049
3500,0.156400,0.123164,0.955080,0.955332,0.960720,0.950004
4000,0.146200,0.142500,0.952080,0.953012,0.971920,0.934826
4500,0.149700,0.125095,0.956640,0.956902,0.962720,0.951154


TrainOutput(global_step=4500, training_loss=0.23293597581651476, metrics={'train_runtime': 5968.5176, 'train_samples_per_second': 83.773, 'train_steps_per_second': 2.62, 'total_flos': 3.80914144441536e+16, 'train_loss': 0.23293597581651476, 'epoch': 5.754475703324808})


{'eval_loss': 0.12316359579563141, 'eval_accuracy': 0.95508, 'eval_f1': 0.9553319279264946, 'eval_recall': 0.96072, 'eval_precision': 0.9500039553832766, 'eval_runtime': 273.4779, 'eval_samples_per_second': 91.415, 'eval_steps_per_second': 2.859, 'epoch': 5.754475703324808}


# 5. QLoRA

We will not be doing experiments using QLoRA due to existing issue pertaining to mixed precision training for DeBERTaV3 (https://github.com/huggingface/transformers/pull/35336). Instead, we will be demostrating the effect of QLoRA in a separate finetuning experiment comparison for BERT model in a separate notebook.